In [1]:
import pickle as p
import numpy as np
import matplotlib.pyplot as plt
import tools as tl
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_curve, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [54]:
def findZeros(input, prefix=""):
    x, y = np.unique(input,return_counts=True)
    hasZeros = len(x[x==0]) > 0
    if(hasZeros) : print(f"{prefix}\n {y[0]} zeros found\n")
    else : print(f"{prefix}\n No zeros found\n")

In [55]:
dataset=p.load(open("pimaDiabetes.p","rb"))

#Data Extraction
target = dataset['target']
data = dataset['data']
feature_names = dataset["feature_names"]

#Data description
print(dataset["DESCR"])

This dataset is originally from the National
Institute of Diabetes and Digestive and Kidney Diseases.
The objective of the dataset is to diagnostically predict
whether or not a patient has diabetes, based on certain
diagnostic measurements included in the dataset.
Several constraints were placed on the selection of these
instances from a larger database. In particular, all patients
here are females at least 21 years old of Pima Indian heritage.
The dataset contains information of 768 women from a population
near Phoenix, Arizona, USA. The outcome tested was Diabetes,
268 tested positive and 500 tested negative.

For more inforamtion:
Smith, J.W., Everhart, J.E., Dickson, W.C., Knowler, W.C., & Johannes, R.S.
(1988). Using the ADAP learning algorithm to forecast the onset of diabetes mellitus.
In Proceedings of the Symposium on Computer Applications and Medical Care (pp. 261--265).
IEEE Computer Society Press.


| Qualidade | Gama de Valores|
| :-: | :-: |
| Número de gravidezes | [0, 17] | 
| Concentração de glicose plasmática em 2 horas num teste oral de tolerância à glicose | [0, 199] |
| Pressão arterial diastólica (mm Hg) | [0, 122] |
| Espessura da dobra cutânea dos tríceps (mm) | [0, 99] |
| Nível sérico de insulina em 2 horas (µh/ml) | [0, 846] |
| Índice de massa corporal (peso em kg/altura em m) | [0, 67.1] |
| Diabetes Pedigree Function | [0.078, 2.42] |
| Idade (anos) | [21, 81] |

## Tratamento de zeros no dataset

In [59]:
for i in range(len(data[0])):
    findZeros(data[:,i],f"({feature_names[i]})")

(Number of times pregnant)
 111 zeros found

(Plasma glucose concentration at 2 Hours in an oral glucose tolerance test (GTIT))
 5 zeros found

(Diastolic Blood Pressure (mm Hg))
 35 zeros found

(Triceps skin fold thickness (mm))
 227 zeros found

(2-Hour Serum insulin (µh/ml))
 374 zeros found

(Body mass index [weight in kg/(Height in m)])
 11 zeros found

(Diabetes pedigree function)
 No zeros found

(Age (years))
 No zeros found



## Divisão do data set
| treino | teste | validação |
| :-: | :-: | :-: | 
| 80% | 10% | 10% |

In [60]:
X2, treino_data, y2, treino_target = train_test_split(data, target, test_size=80/100, random_state=42, stratify=target)
valid_data, teste_data, valid_target, teste_target = train_test_split(X2,y2, test_size=1/2, random_state=42, stratify=y2)

print("treino_data",treino_data.shape, "\ttreino_target",treino_target.shape)
print("valid_data",valid_data.shape, "\tvalid_target",valid_target.shape)
print("teste_data",teste_data.shape, "\ttest_target",teste_target.shape)

# [:,0] - n gravidez
# [:,1] - glicose
# [:,2] - pressao arterial
# [:,3] - dobra do tricep
# [:,4] - nivel de insulina
# [:,5] - IMC
# [:,6] - DPF
# [:,7] - idade

treino_data (615, 8) 	treino_target (615,)
valid_data (76, 8) 	valid_target (76,)
teste_data (77, 8) 	test_target (77,)


In [61]:
#Normalização do conjunto de treino
norm_treino_data = tl.normalize_data(treino_data)

#Normalização do conjunto de teste
norm_teste_data = tl.normalize_data(teste_data)

#Normalização do conjunto de validação
norm_valid_data = tl.normalize_data(valid_data)

In [57]:
# Grelha de parâmetros com os vários parâmetros que vão ser utilizados para encontrar a melhor combinação de parâmetros
lr_param_grid = {
    'C': [0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 5.5, 6, 6.5, 7, 7.5, 8, 8.5, 9, 9.5, 10],
    'max_iter': [300, 400, 500, 600, 700, 800, 900, 1000]
}

# Inicializar o classificador
lr = LogisticRegression(random_state=42)

lr_params = tl.get_best_params(lr_param_grid, lr, treino_data, treino_target)
print(lr_params)

lr = LogisticRegression(C=lr_params['C'], max_iter=lr_params['max_iter'], random_state=42)
lr = lr.fit(treino_data, treino_target)

{'C': 1.5, 'max_iter': 300}


In [62]:
lr_pred = lr.predict(teste_data)

tl.predition_stats(teste_target,lr_pred)


True Positives 48 (96.0%)            
False Negatives 2 (4.0%)            

True Negatives 20 (74.0%)            
False Positives 7 (26.0%)            

Recall Rate: 96.0%            
Specificity Rate: 74.1%            

Positive Precision : 87.3%            
Negative Precision : 90.9%            

F-Score : 91.4435%            
G-Score : 91.5467%
